In [43]:
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

C:\Program Files\Miniconda3\envs\ai\lib\site-packages\PIL\Image.py:125: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 9.5.0
Pillow version: 9.4.0
  warnings.warn(str(v), RuntimeWarning)


ImportError: The _imaging extension was built for another version of Pillow or PIL:
Core version: 9.5.0
Pillow version: 9.4.0

In [80]:
REWARDS = np.array([
    [0, 100],
    [100, 500],
    [500, 1000],
    [1000, 5000],
    [5000, 10000],
    [10000, 50000],
    [50000, 100000],
    [100000, 500000],
    [500000, 1000000],
    [1000000, 5000000]
])
# REWARDS = np.array([
#     [500, 100],
#     [1000, 500],
#     [5000, 1000],
#     [10000, 5000],
#     [50000, 10000],
#     [100000, 50000],
#     [500000, 100000],
#     [1000000, 500000],
#     [5000000, 1000000],
#     [10000000, 5000000]
# ])
PROBABILITIES = np.array([
    0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1
])
REWARDS, PROBABILITIES

(array([[      0,     100],
        [    100,     500],
        [    500,    1000],
        [   1000,    5000],
        [   5000,   10000],
        [  10000,   50000],
        [  50000,  100000],
        [ 100000,  500000],
        [ 500000, 1000000],
        [1000000, 5000000]]),
 array([0.99, 0.9 , 0.8 , 0.7 , 0.6 , 0.5 , 0.4 , 0.3 , 0.2 , 0.1 ]))

In [81]:
class ACTION(Enum):
    PLAY = 0
    QUIT = 1

In [82]:
class Game:
    def __init__(self):
        self.round = 0
        self.reward = 0
        self.finished = False

    def play_move(self, action):
        if action == ACTION.QUIT:
            self.finished = True
        elif action == ACTION.PLAY:
            if np.random.random() < PROBABILITIES[self.round]:
                self.reward += REWARDS[self.round][1]
            else:
                self.reward -= REWARDS[self.round][0]
                self.finished = True

            self.round += 1
            if self.round == 10:
                self.finished = True

In [83]:
game = Game()

In [84]:
def simulate_game():
    game = Game()
    while not game.finished:
        game.play_move(ACTION.PLAY)
    return game.reward, game.round

In [85]:
def monte_carlo_simulation(n):
    rewards = np.zeros(n)
    rounds = np.zeros(n)
    for i in range(n):
        rewards[i], rounds[i] = simulate_game()

    # bins = np.arange(0, 10, 1)
    # plt.hist(rounds, bins=bins)
    return np.mean(rounds), np.std(rounds), np.min(rounds), np.max(rounds), np.median(rounds)

In [92]:
monte_carlo_simulation(100000)

(4.61713, 1.7382579104091544, 1.0, 10.0, 4.0)

In [88]:
avg_win_rewards = np.zeros((11, 2))

for i in range(1, 11):
    avg_win_rewards[i, 0] += np.sum(REWARDS[:i, 1])
    avg_win_rewards[i, 0] *= np.product(PROBABILITIES[:i])
    avg_win_rewards[i, 1] += np.sum(REWARDS[:i-1, 1]) - REWARDS[i-1, 0]
    avg_win_rewards[i, 1] *= np.product(PROBABILITIES[:i-1]) * (1 - PROBABILITIES[i-1])

avg_win_rewards

array([[    0.        ,     0.        ],
       [   99.        ,     0.        ],
       [  534.6       ,     0.        ],
       [ 1140.48      ,    17.82      ],
       [ 3293.136     ,   128.304     ],
       [ 4969.6416    ,   319.3344    ],
       [ 9969.2208    ,   987.9408    ],
       [ 9975.20832   ,  1490.89248   ],
       [11973.842496  ,  2791.381824  ],
       [ 5987.2804992 ,  2394.0499968 ],
       [ 2394.98404992,  2155.29164928]])

In [89]:
avg_rewards = np.zeros(11)
avg_rewards[0] = avg_win_rewards[0, 0]

for i in range(1, 11):
    avg_rewards[i] = avg_win_rewards[i, 0] + avg_win_rewards[i, 1] + np.sum(avg_win_rewards[:i, 1])

avg_rewards

array([    0.    ,    99.    ,   534.6   ,  1158.3   ,  3439.26  ,
        5435.1   , 11422.62  , 12919.5   , 17709.516 , 14117.004 ,
       12679.9992])

In [90]:
np.argmax(avg_rewards)

8